In [46]:
from haystack.document_stores import OpenSearchDocumentStore

doc_store = OpenSearchDocumentStore(embedding_dim=384)

/Users/tuanacelik/opt/anaconda3/envs/sagemeker/lib/python3.11/site-packages/opensearchpy/connection/http_urllib3.py:199: UserWarning: Connecting to https://localhost:9200 using SSL with verify_certs=False is insecure.
  warnings.warn(
/Users/tuanacelik/opt/anaconda3/envs/sagemeker/lib/python3.11/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/tuanacelik/opt/anaconda3/envs/sagemeker/lib/python3.11/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/tuanacelik/opt/anaconda3/envs/sagemeker/lib/python3.11/site-packages/urllib3

In [48]:
doc_store.get_all_documents()[0]

/Users/tuanacelik/opt/anaconda3/envs/sagemeker/lib/python3.11/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/tuanacelik/opt/anaconda3/envs/sagemeker/lib/python3.11/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/tuanacelik/opt/anaconda3/envs/sagemeker/lib/python3.11/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-war

<Document: {'content': 'To hide the Layers panel, select the collapse (arrow) icon in the panel’s upper-right corner.\nLayer settings\nTo change the default map settings, select Default map in the Layers panel. Under Layer settings, you can change the layer name and description and configure zoom levels and opacity for your layer: Zoom levels: By default, a layer is visible at all zoom levels. ', 'content_type': 'text', 'score': None, 'meta': {'_split_id': 4, '_split_overlap': [{'range': [0, 184], 'doc_id': '1cde4a4adc1a6f4fbfbe3a1df1b641e2'}, {'range': [185, 370], 'doc_id': '8f3ee7c2ab540754d4f9b6c2031868b7'}], 'title': 'Using maps', 'ancestors': ['OpenSearch Dashboards', 'Building data visualizations', 'Using coordinate and region maps'], 'type': 'DOCS', 'url': '/docs/latest/dashboards/visualize/maps/'}, 'id_hash_keys': ['content'], 'embedding': None, 'id': '4e0c9c126ecd27253b91c61c792643fd'}>

In [16]:
from haystack.nodes import EmbeddingRetriever, PromptTemplate, PromptNode

retriever = EmbeddingRetriever(document_store=doc_store, embedding_model="sentence-transformers/all-MiniLM-L12-v2", devices=["mps"])


In [20]:
doc_store.update_embeddings(retriever)

/Users/tuanacelik/opt/anaconda3/envs/sagemeker/lib/python3.11/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
Updating embeddings:   0%|          | 0/8841 [00:00<?, ? Docs/s]/Users/tuanacelik/opt/anaconda3/envs/sagemeker/lib/python3.11/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/tuanacelik/opt/anaconda3/envs/sagemeker/lib/python3.11/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: http

In [10]:
from getpass import getpass

api_key = getpass("Enter model provider API key:")

In [11]:
gen_qa_with_references = PromptNode(default_prompt_template="question-answering-with-references", model_name_or_path="gpt-4", api_key=api_key)

/Users/tuanacelik/opt/anaconda3/envs/sagemeker/lib/python3.11/site-packages/haystack/nodes/prompt/prompt_template.py:421: UserWarning: You're using a legacy prompt template 'question-answering-with-references', we strongly suggest you use prompts from the official Haystack PromptHub: https://prompthub.deepset.ai/
  warnings.warn(


In [12]:
from haystack import Pipeline

pipe = Pipeline()
pipe.add_node(component=retriever, name="Retriever", inputs=['Query'])
pipe.add_node(component=gen_qa_with_references, name="GenQAWithRefPromptNode", inputs=["Retriever"])

In [41]:
from haystack.utils import print_answers

result = pipe.run("How can I install the opensearch client?", params={"Retriever":{"top_k": 5}})

Batches: 100%|██████████| 1/1 [00:00<00:00,  1.14it/s]
/Users/tuanacelik/opt/anaconda3/envs/sagemeker/lib/python3.11/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [43]:
print_answers(results=result, details="minimum")

'Query: How can I install the opensearch client?'
'Answers:'
[   {   'answer': 'Install the opensearch client using pip: for opensearch-py, '
                  'use "pip install opensearch-py" (Document[2]); for '
                  'opensearch-dsl, use "pip install opensearch-dsl" '
                  '(Document[5]).'}]
